In [1]:
from pymoo.termination.default import DefaultMultiObjectiveTermination, DefaultSingleObjectiveTermination
from pymoo.algorithms.moo.nsga2 import NSGA2
from mutation import MyMutation
from crossover import TraceCrossover
from encoder import Encoder
from sampling import MySampling
from callback import UpdatePopulationCallback
from tools import Tools
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import warnings
import random
import pandas as pd
import logging
from terminator import MyTermination
from pymoo.algorithms.soo.nonconvex.ga import GA
from problem import Problem_single_ElementWise, Problem_multi_ElementWise
import time
import itertools
from pymoo.optimize import minimize

In [2]:
logging.getLogger('matplotlib').setLevel(logging.WARNING)
warnings.filterwarnings("ignore", ".*feasible.*")


pop_list = [2000, 3000]  # population sizes
num_event_list = [20, 30, 50]  # trace lengths
declare_model_list = ["base_model.decl"]  # declare models
termination_list = ["my_termination", "multi", "single"]  # termination criteria


activities_name = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T"]

In [3]:


def setup_initial_population(trace_length, n_traces, activities_name, encoder):
    """
    Sets up the initial population and shared parameters for the given trace length.
    """

    initial_population = [[random.choice(activities_name) for _ in range(trace_length)] for _ in range(n_traces)]
    initial_encoded_pop = [encoder.encode(trace) for trace in initial_population]
    variable_boundaries = [1] * trace_length
    features_range = Tools.calculate_feature_range(initial_encoded_pop, variable_boundaries)
    lower_bounds = [x[0] for x in features_range]
    upper_bounds = [x[1] for x in features_range]

    mutation = MyMutation(feature_range=features_range)
    crossover = TraceCrossover(variable_boundaries=variable_boundaries)
    sampling = MySampling(initial_population=initial_encoded_pop)

    return initial_population, initial_encoded_pop, features_range, variable_boundaries, lower_bounds, upper_bounds, mutation, crossover, sampling

def initialize_shared_components(activities_name, path_to_declareModel):


    # generate timestamps and prepare data
    timestamps = Tools.generate_random_timestamps(len(activities_name))
    case_concept_name = ['1'] * len(activities_name)

    data = {
        'case:concept:name': case_concept_name,
        'concept:name': activities_name,
        'timestamp': timestamps,
    }

    dataframe = pd.DataFrame(data)
    dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'])

    # initialize encoder, declare model, and event log
    encoder = Encoder(activities_name)
    declare = DeclareModel().parse_from_file(path_to_declareModel)

    event_log = D4PyEventLog()
    event_log.log = dataframe
    event_log.timestamp_key = "timestamp"
    event_log.activity_key = "concept:name"

    return encoder, declare, event_log, dataframe


with open("test_results.csv", "w") as f:

    f.write("ID,Population,TraceLength,Model,Termination,Algorithm,ExecutionTime\n")

    ID = 1

    # loop through all combinations
    for combination in itertools.product(pop_list, num_event_list, declare_model_list, termination_list):
        pop_size, trace_length, model, termination = combination

        print(f"Running with ID={ID}: Population={pop_size}, TraceLength={trace_length}, Model={model}, Termination={termination}")


        start_time = time.time()



        # set up initial population and shared parameters
        (
            encoder,
            declare,
            event_log,
            dataframe
         ) = initialize_shared_components(activities_name=activities_name, path_to_declareModel="declare_models/" + model)

        (
            initial_population,
            initial_encoded_pop,
            features_range,
            variable_boundaries,
            lower_bounds,
            upper_bounds,
            mutation,
            crossover,
            sampling
        ) = setup_initial_population(trace_length, n_traces=10, activities_name=activities_name, encoder=encoder)


        if termination == 'my_termination': # run BOTH single-objective and multi-objective with my_termination


            my_termination = MyTermination(n_required=pop_size)  # set the termination criteria of pop_size and increment the pop_size
            pop_size_for_terminator = int(pop_size * 1.5)

            for algorithm_type in ['single', 'multi']:
                if algorithm_type == 'single':
                    problem = Problem_single_ElementWise(
                        trace_length=trace_length,
                        encoder=encoder,
                        d4py=declare,
                        initial_population=initial_encoded_pop,
                        xl=lower_bounds, xu=upper_bounds,
                        event_log=event_log,
                        dataframe=dataframe
                    )
                    algorithm = GA(
                        problem=problem,
                        pop_size=pop_size_for_terminator,
                        sampling=sampling,
                        crossover=crossover,
                        mutation=mutation,
                        callback=UpdatePopulationCallback(problem=problem, plot=0),
                        eliminate_duplicates=False,
                    )
                elif algorithm_type == 'multi':
                    problem = Problem_multi_ElementWise(
                        trace_length=trace_length,
                        encoder=encoder,
                        d4py=declare,
                        initial_population=initial_encoded_pop,
                        xl=lower_bounds, xu=upper_bounds,
                        event_log=event_log,
                        dataframe=dataframe
                    )
                    algorithm = NSGA2(
                        problem=problem,
                        pop_size=pop_size_for_terminator,
                        sampling=sampling,
                        crossover=crossover,
                        mutation=mutation,
                        callback=UpdatePopulationCallback(problem=problem, plot=0),
                        eliminate_duplicates=False,
                    )

                # run the optimization for "my_termination"
                try:
                    result = minimize(problem, algorithm, termination=my_termination, verbose=False)
                    exec_time = time.time() - start_time

                    # Log results
                    f.write(f"{ID},{pop_size},{trace_length},{model},{termination},{algorithm_type},{exec_time:.2f}\n")

                    print(f"Execution Time ({algorithm_type}): {exec_time:.2f} seconds")
                    ID += 1
                except Exception as e:
                    print(f"Error encountered with configuration {combination} ({algorithm_type}): {e}")
                    f.write(f"{ID},{pop_size},{trace_length},{model},{termination},{algorithm_type},ERROR\n")

                    ID += 1

        elif termination == 'multi':

            selected_termination = DefaultMultiObjectiveTermination(
                xtol=1e-8,
                cvtol=1e-6,
                ftol=0.0025,
                period=30,
                n_max_gen=1000,
                n_max_evals=100000,
            )
            problem = Problem_multi_ElementWise(
                trace_length=trace_length,
                encoder=encoder,
                d4py=declare,
                initial_population=initial_encoded_pop,
                xl=lower_bounds, xu=upper_bounds,
                event_log=event_log,
                dataframe=dataframe
            )
            algorithm = NSGA2(
                problem=problem,
                pop_size=pop_size,
                sampling=sampling,
                crossover=crossover,
                mutation=mutation,
                callback=UpdatePopulationCallback(problem=problem, plot=0),
                eliminate_duplicates=False,
            )

            # run the optimization
            try:
                result = minimize(problem, algorithm, termination=selected_termination, verbose=False)
                exec_time = time.time() - start_time

                # log results
                f.write(f"{ID},{pop_size},{trace_length},{model},{termination},multi,{exec_time:.2f}\n")
                print(f"Execution Time (multi): {exec_time:.2f} seconds")
                ID += 1
            except Exception as e:
                print(f"Error encountered with configuration {combination} (multi): {e}")
                f.write(f"{ID},{pop_size},{trace_length},{model},{termination},multi,ERROR\n")
                ID += 1

        elif termination == 'single':

            selected_termination = DefaultSingleObjectiveTermination(
                xtol=1e-8,
                cvtol=1e-6,
                ftol=1e-6,
                period=20,
                n_max_gen=1000,
                n_max_evals=100000,
            )
            problem = Problem_single_ElementWise(
                trace_length=trace_length,
                encoder=encoder,
                d4py=declare,
                initial_population=initial_encoded_pop,
                xl=lower_bounds, xu=upper_bounds,
                event_log=event_log,
                dataframe=dataframe
            )
            algorithm = GA(
                problem=problem,
                pop_size=pop_size,
                sampling=sampling,
                crossover=crossover,
                mutation=mutation,
                callback=UpdatePopulationCallback(problem=problem, plot=0),
                eliminate_duplicates=False,
            )

            # run the optimization
            try:
                result = minimize(problem, algorithm, termination=selected_termination, verbose=False)
                exec_time = time.time() - start_time

                # log results
                f.write(f"{ID},{pop_size},{trace_length},{model},{termination},single,{exec_time:.2f}\n")
                print(f"Execution Time (single): {exec_time:.2f} seconds")

                ID += 1
            except Exception as e:
                print(f"Error encountered with configuration {combination} (single): {e}")
                f.write(f"{ID},{pop_size},{trace_length},{model},{termination},single,ERROR\n")

                ID += 1


Running with ID=1: Population=2000, TraceLength=20, Model=base_model.decl, Termination=my_termination
Execution Time (single): 29.27 seconds
Execution Time (multi): 60.93 seconds
Running with ID=3: Population=2000, TraceLength=20, Model=base_model.decl, Termination=multi
Execution Time (multi): 76.00 seconds
Running with ID=4: Population=2000, TraceLength=20, Model=base_model.decl, Termination=single
Execution Time (single): 46.43 seconds
Running with ID=5: Population=2000, TraceLength=30, Model=base_model.decl, Termination=my_termination
Execution Time (single): 33.89 seconds
Execution Time (multi): 68.43 seconds
Running with ID=7: Population=2000, TraceLength=30, Model=base_model.decl, Termination=multi
Execution Time (multi): 83.27 seconds
Running with ID=8: Population=2000, TraceLength=30, Model=base_model.decl, Termination=single
Execution Time (single): 73.42 seconds
Running with ID=9: Population=2000, TraceLength=50, Model=base_model.decl, Termination=my_termination
Execution Ti